In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [5]:
model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2574 - accuracy: 0.9260 - val_loss: 0.1412 - val_accuracy: 0.9582
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1148 - accuracy: 0.9671 - val_loss: 0.0965 - val_accuracy: 0.9706
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0790 - accuracy: 0.9763 - val_loss: 0.0917 - val_accuracy: 0.9722
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0590 - accuracy: 0.9815 - val_loss: 0.0781 - val_accuracy: 0.9757
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0440 - accuracy: 0.9865 - val_loss: 0.0806 - val_accuracy: 0.9763
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0352 - accuracy: 0.9887 - val_loss: 0.0803 - val_accuracy: 0.9767
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0282 - accuracy: 0.9913 - val_loss: 0.0797 - val_accuracy:

In [6]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0858 - accuracy: 0.9759


In [7]:
class ClassAccuracy(tf.keras.metrics.Metric):

    def __init__(self, name='class_accuracy', **kwargs):
        super(ClassAccuracy, self).__init__(name=name, **kwargs)

    def update_state(self, y_true, y_pred):
        K = len(set(y_test))
        oyp = tf.one_hot(tf.argmax(y_pred, axis=1), K)
        oyt = tf.one_hot(y_test, K)
        accuracies = tf.reduce_mean(tf.cast(oyp == oyt, tf.float32), axis=0)
        self.accuracies = accuracies
    
    def result(self):
        return self.accuracies

In [8]:
y_pred = model.predict(x_test)

In [22]:
print(tf.one_hot(tf.argmax(y_pred, axis=1), 10))

tf.Tensor(
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(10000, 10), dtype=float32)


In [9]:
print(y_test)

[7 2 1 ... 4 5 6]


In [14]:
tf.cast(y_test == tf.argmax(y_pred, axis=1), tf.float16)

<tf.Tensor: shape=(10000,), dtype=float16, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float16)>

In [16]:
tf.reduce_mean(tf.cast(y_test == tf.argmax(y_pred, axis=1), tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.9759>

In [18]:
tf.reduce_mean(tf.cast(tf.one_hot(y_test, len(set(y_test))) == tf.one_hot(tf.argmax(y_pred, axis=1), 10), tf.float32), axis=0)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.9969, 0.9964, 0.9951, 0.9944, 0.9946, 0.9944, 0.9963, 0.995 ,
       0.9946, 0.9941], dtype=float32)>